In [1]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature=0)
query = "인프런에는 어떤 강의가 있나요?"
llm.invoke(query)

AIMessage(content='인프런은 다양한 주제의 온라인 강의를 제공하는 플랫폼입니다. 주로 IT, 프로그래밍, 데이터 과학, 디자인, 마케팅, 비즈니스 등 여러 분야의 강의를 찾을 수 있습니다. 예를 들어, 다음과 같은 강의들이 있습니다:\n\n1. **프로그래밍**: Python, Java, JavaScript, C++, 웹 개발, 모바일 앱 개발 등.\n2. **데이터 과학**: 데이터 분석, 머신러닝, 인공지능, 데이터 시각화 등.\n3. **디자인**: UI/UX 디자인, 그래픽 디자인, 포토샵, 일러스트레이터 등.\n4. **마케팅**: 디지털 마케팅, SEO, 소셜 미디어 마케팅 등.\n5. **비즈니스**: 스타트업, 경영, 재무, 프로젝트 관리 등.\n\n각 강의는 강사에 따라 다르며, 초급부터 고급 과정까지 다양하게 제공됩니다. 관심 있는 분야에 맞는 강의를 찾아보시면 좋을 것 같습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 222, 'prompt_tokens': 18, 'total_tokens': 240, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run-e87c9bc0-22dc-4569-941c-d54a8f9ed7d4-0', usage_metadata={'in

In [8]:
from langchain_core.prompts import PromptTemplate



In [2]:
%pip install -qU beautifulsoup4

You should consider upgrading via the '/Users/a202304035/.pyenv/versions/inflearn-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.inflearn.com/courses")

In [16]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500, # chunk 하나가 가질 수 있는 token 수
    chunk_overlap = 200 # chunk 간 token 을 겹치게 하는 범위 (유사도 검색의 성능을 더 올림)
)


document_list = loader.load_and_split(text_splitter=text_splitter)

In [17]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')
database = Chroma.from_documents(documents=document_list, embedding= embedding,
     persist_directory='./inflearn-lectures', collection_name='inflearn-lectures')

In [18]:
retriever = database.as_retriever(search_kwargs = {'k':1})
question = "인프런에는 어떤 데이터과학 관련 강의들이 있나요?"

retrieved_docs = retriever.invoke(question)
retrieved_docs

[Document(metadata={'description': '프로그래밍, 인공지능, 데이터, 마케팅, 디자인등 입문부터 실전까지 업계 최고 선배들에게 배울 수 있는 곳.', 'language': 'ko', 'source': 'https://www.inflearn.com/courses/data-science', 'title': '인프런 - 라이프타임 커리어 플랫폼'}, page_content='인프런 - 라이프타임 커리어 플랫폼인프런 - 라이프타임 커리어 플랫폼검색검색전체전체기술 검색기술 검색답변 활발미션모임/부트캠프난이도난이도추천순추천순')]

In [9]:
rag_prompt_template = PromptTemplate(
    template="""
Your are a helpful assistant that can answer questions about the Inflearn Websites.
You are given the following context:
{context}
Question : {question}
""",
input_variables=["context","question"]
)

rag_chain = rag_prompt_template | llm
rag_chain.invoke({'context':retrieved_docs, 'question':question})

AIMessage(content='인프런에서는 데이터 과학 관련 다양한 강의를 제공합니다. 이 강의들은 입문부터 실전까지 다양한 수준의 내용을 포함하고 있으며, 프로그래밍, 인공지능, 데이터 분석 등 여러 주제를 다룹니다. 구체적인 강의 목록은 인프런 웹사이트의 데이터 과학 카테고리에서 확인할 수 있습니다. 각 강의는 업계 최고 선배들로부터 배울 수 있는 기회를 제공합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 189, 'total_tokens': 284, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'stop', 'logprobs': None}, id='run-4af37491-da63-4f64-a2bf-16fbb1e64c00-0', usage_metadata={'input_tokens': 189, 'output_tokens': 95, 'total_tokens': 284, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
from langchain import hub
rag_prompt_template = hub.pull('rlm/rag-prompt')

In [11]:
langchaing_prompt_rag_chain = rag_prompt_template | llm
langchaing_prompt_rag_chain.invoke({'context':retrieved_docs,'question':question})

AIMessage(content='인프런에서는 데이터 과학 관련 다양한 강의를 제공하고 있습니다. 이 강의들은 입문부터 실전까지 다양한 수준의 내용을 포함하고 있으며, 프로그래밍, 인공지능, 데이터 분석 등 여러 주제를 다룹니다. 구체적인 강의 목록은 인프런 웹사이트의 데이터 과학 카테고리에서 확인할 수 있습니다. 각 강의는 업계 최고 선배들로부터 배울 수 있는 기회를 제공합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 189, 'total_tokens': 286, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'stop', 'logprobs': None}, id='run-16434a44-723e-4691-a62a-54ceee3bb717-0', usage_metadata={'input_tokens': 189, 'output_tokens': 97, 'total_tokens': 286, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context":retriever, "question":RunnablePassthrough()} 
    | rag_prompt_template
    | llm
)
rag_chain.invoke(question)

AIMessage(content='인프런에서는 데이터 과학 관련 다양한 강의를 제공하고 있습니다. 강의는 입문부터 실전까지 다양한 수준으로 구성되어 있으며, 프로그래밍, 인공지능, 데이터 분석 등 여러 주제를 다룹니다. 구체적인 강의 목록은 인프런 웹사이트의 데이터 과학 카테고리에서 확인할 수 있습니다. 각 강의는 업계 최고 선배들로부터 배울 수 있는 기회를 제공합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 189, 'total_tokens': 284, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2a752fb-8992-49f8-83c1-60626dc81417-0', usage_metadata={'input_tokens': 189, 'output_tokens': 95, 'total_tokens': 284, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})